In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow_datasets as tfds
from time import time

In [2]:
from preprocess import process_dir_images, preprocess_lfw
from predict import predict_on_dataset

In [2]:

# load LFW dataset from Tensorflow Datasets
ds, info = tfds.load('lfw', split='train', with_info=True, as_supervised=True)


In [3]:
for label, image in ds.take(1):
    print(image.shape, '\n' ,label.numpy())

(250, 250, 3) 
 b'Tom_Amstutz'


In [4]:
# preprocess the dataset
def preprocess_images(label, image):
    # If image is 2D (grayscale), add channel dimension
    if len(image.shape) == 2:
        image = tf.expand_dims(image, axis=-1) 
    # If image is not RGB, convert to RGB    
    if image.shape[-1] != 3:
        image = tf.image.grayscale_to_rgb(image)
    # Resize the image to a fixed size
    image = tf.image.resize(image, [128, 128])
    
    # Normalize the pixel values to [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    
    return image, 1

In [5]:

face_ds = ds.map(preprocess_images, num_parallel_calls=tf.data.AUTOTUNE).shuffle(35000)
#len = len(list(face_ds))
face_ds = face_ds.take(3000)  # limiting to 2000 samples for faster training

In [6]:
for image, label in face_ds.take(1):
    print(image.shape, '\n', label.numpy())

(128, 128, 3) 
 1


In [ ]:
num_random_images = 100 # number of random images to be generated form non face data

# Generate a dataset of random images
random_images_ds = tf.data.Dataset.from_tensor_slices(
    tf.random.uniform(shape=(num_random_images, 128, 128, 3), dtype=tf.float32)
)

# Generate a dataset of zero labels
zero_labels_ds = tf.data.Dataset.from_tensor_slices(
    tf.zeros(shape=(num_random_images,), dtype=tf.int32)
)
# Combine random images and labels
noface_ds = tf.data.Dataset.zip((random_images_ds, zero_labels_ds))


In [47]:
#for image, label in noface_ds.take(1):
  #  print(image.numpy(), '\n', label.numpy())

In [3]:
# load no-face images from a directory

dir_images = tf.keras.utils.image_dataset_from_directory(
    "E:/AI-ML-DL/python projects/face detection/Data/Training/", image_size=(128, 128), 
    label_mode='int', shuffle=True, 
    batch_size= None, 
    class_names=['no_face', 'face']
    )

#for image, label in dir_images.take(1):
#    print(image.numpy(), '\n', label.numpy())



Found 1013 files belonging to 2 classes.


In [23]:
import matplotlib.pyplot as plt
print("class names of loaded from directory data: ", dir_images.class_names)
img, label = next(iter(dir_images)) # making dataset iterable then taking one sample
print("the label for loaded dir image is: ",label.numpy())
print("The shape of dir image is : ",img.shape)

#plt.figure()
#plt.imshow(img.numpy().astype("uint8"))

#for _, label in dir_images.take(1):
#    print([int(i) for i in label.numpy()])  # this method will work only on batched data , just print label.numpy() for unbatched data

class names of loaded from directory data:  ['no_face', 'face']
the label for loaded dir image is:  0
The shape of dir image is :  (128, 128, 3)


In [ ]:
print(getattr(dir_images, 'class_names', 'lol'))  # it will print class names because it's dataframe object which have attributes
print(getattr(face_ds, 'class_names', 'lol'))   # but as soon as we operate or map a function and create now object it loose attributes just data and labels 

In [12]:
def process_dir_images(image, label):
    # If image is 2D (grayscale), add channel dimension
    if len(image.shape) == 2:
        image = tf.expand_dims(image, axis=-1) 
    # If image is not RGB, convert to RGB    
    if image.shape[-1] != 3:
        image = tf.image.grayscale_to_rgb(image)
    # Resize the image to a fixed size
    image = tf.image.resize(image, [128, 128])
    
    # Normalize the pixel values to [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    
    return image, label

In [4]:
local_ds = dir_images.map(process_dir_images, num_parallel_calls=tf.data.AUTOTUNE)
type(local_ds)

tensorflow.python.data.ops.map_op._ParallelMapDataset

In [ ]:
#for image, label in local_ds.take(1):
#    print(image.numpy(), '\n', label.numpy())
#    print(label)

In [9]:
# Combine all datasets here or as you wish
dataset = face_ds.concatenate(local_ds).shuffle(1100)
#dataset = local_ds.shuffle(1100)

dataset_len = len(list(dataset))
train_size = int(0.8*dataset_len)

train_ds = dataset.take(train_size).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)  # creating batch is necessary for training and prefretching helps in performance(only with GPU)
test_ds = dataset.skip(train_size).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
len(next(iter(train_ds)))

In [ ]:
count1 = 0
count0 = 0
for image, label in train_ds:
    for lab in label:
         if lab ==1: 
             # print(image.shape, '\n', label.numpy())
             count1 += 1
         else:
             count0 += 1

print(f"Number of images with face: {count1}, Number of images without face: {count0}")

In [ ]:
from tensorflow.keras.optimizers import Adam
model = Sequential([

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='tanh', input_shape=(128,128,3)), # use activation "tanh"
    MaxPooling2D(2,2),
    Dropout(0.3),



    Flatten(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')])

# optimizer 
optimizer = Adam(learning_rate=0.005)  # you can adjust the learning rate as needed

# compile the model
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
# train the model
model.fit(train_ds, epochs=5, validation_data=test_ds, verbose=1)

In [ ]:
#del model    # delete model to retrain if want to retrain if needed to restart fresh model
#tf.keras.backend.clear_session()  # clear the session to free up resources(new method)
#tf.compat.v1.reset_default_graph   # clear the session to free up resources(old method)

In [ ]:
# save model to a file
#model.save('E:/AI-ML-DL/saved models/face_detection_model.h5')

In [7]:
# load saved moedl to continue training or evaluation   

from tensorflow.keras.models import load_model

# Load the model
model = load_model('E:/AI-ML-DL/saved models/face_detection_model.h5')

# Optional: compile if you want to continue training
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Optional: continue training
#model.fit(dataset, epochs=10, validation_data=dataset, verbose=1)

In [8]:
# function to process loaded single image and make prediction
def predict_face(image_path):
    img = load_img(image_path, target_size=(128,128))
    img = img_to_array(img, dtype='float32')/ 255.0
    img = np.expand_dims(img, axis=0) # add batch dimension because model expects a batch of images
    prediction = model.predict(img)
    print(f"Prediction shape: {prediction}")
    return prediction[0][0]   # return the predicted probability for the face presence, was shped like [[0.8]] which is 2d array

In [ ]:
# predict using the model

# Single image prediction
image_path = "E:/AI-ML-DL/python projects/face detection/Data/Testing/object3.jpg"  # single image path
prediction = predict_face(image_path)
if prediction >0.5:
    print("The image is a face.")
else:
    print("The image is not a face.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Prediction shape: [[0.19721028]]
The image is not a face.


In [3]:
# load testing dataset from a local directory
# give the path with two sub folders: 'face' and 'no_face'
test_data = tf.keras.utils.image_dataset_from_directory('E:/AI-ML-DL/python projects/face detection/Data/Testing', image_size=(128, 128), shuffle=False, batch_size=None)

# process the images in the test_data
test_images = test_data.map(process_dir_images, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)

Found 144 files belonging to 2 classes.


In [ ]:
#pred = model.predict(test_images)
pred = model.predict(test_images)


In [ ]:
print(pred.shape)
print(pred)
#pred = np.where(pred > 0.5, 1, 0) #

In [ ]:
yes = 0
no = 0 
for pre in pred[:, 0]:  # Assuming pred is a 2D array with shape (num_samples, 1)
    if pre > 0.5:
        print(pre)
        yes += 1
    else:
        no += 1
print(f"Number of faces detected: {yes}")
print(f"Number of non-faces detected: {no}")

In [ ]:
from predict import predict_on_dataset
num_batches = 2
predict_on_dataset(test_images, num_batches)